# 삼성 DS-KAIST AI Expert 프로그램
## Week 3: 신경망 기초, 다층신경망

실습 일시: 2019년 7월 15일 (월), 13:30 - 17:30

담당 조교: 정종헌 (jongheonj@kaist.ac.kr)

## Introduction

본 실습에서는 TensorFlow 프레임워크의 전반적인 원리를 이해하고, 이를 이용해 강의에서 소개된 다층신경망을 실습해 보는 것을 목표로 합니다. 실습은 크게 2개의 파트로 구성됩니다: 첫 번째 파트에서는, 연산 그래프를 구성하고 그것의 그래디언트를 구하는 과정을 통해 TensorFlow의 자동 미분 기능을 이해할 것입니다. 두 번째 파트에서는, 간단한 이미지 인식 문제를 해결하는 TensorFlow 신경망을 모델링하고 학습 및 테스트하는 과정을 경험해볼 것입니다. 기초적인 선형 모델에서부터 시작하여 컨볼루션 신경망 모델까지, 학습 모델을 변경해 가면서 인식 성능을 비교해보고, 파라미터 수 대비 인식 성능을 높이기 위한 방법을 고민해볼 것입니다.

In [ ]:
# 실습에서 사용할 모듈을 import 합니다.

# IPython 상에서 matplotlib 라이브러리가 
# 별도의 창 없이 즉시 plot을 출력할 수 있도록 설정을 변경합니다.
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

# "즉시 실행" (eager execution) 기능을 활성화합니다.
# 즉시 실행 활성화로 TensorFlow를 대화형 프론트엔드(frontend)에 가깝게 만들어 줍니다.
# 해당 기능은 TensorFlow 2.0 부터는 기본적으로 활성화됩니다. 
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

## Part 1: Tensorflow를 활용한 역전파 알고리즘 수행

TensorFlow와 같은 머신러닝 프레임워크와 일반적인 컴퓨팅 라이브러리와 가장 큰 차이점 중 하나는 __자동 미분__ (automatic differentiation; 주어진 입력 변수에 대한 연산의 그래디언트를 계산하는 것)의 지원 유무입니다. TensorFlow 프레임워크에서 구현되는 대부분의 함수는 임의의 입력과 출력에 대한 미분값을 계산할 수 있기 때문에, 현대 딥러닝 모델의 학습과정의 중요한 요소인 함수의 그래디언트를 쉽게 얻어낼 수 있습니다. 본 실습의 Part 1에서는 TensorFlow의 자동 미분기능을 다양한 환경에서 테스트함으로써 역전파 알고리즘의 동작을 확인하고, 딥러닝 모델이 학습되는 원리를 이해하는 것을 목표로 합니다.


In [ ]:
# 실습 과정에서 사용되는 도움함수를 정의합니다. 

def plot_df_1d(f, df_dx, d2f_dx2=None):
    """일변수 함수에 한하여 도함수를 시각화함"""
    
    xs = np.linspace(-5, 5)
    fs = [f(x) for x in xs]
    dfs = [df_dx(x) for x in xs]
    if d2f_dx2:
        d2fs = [d2f_dx2(x) for x in xs]
        figsize = (18, 5)
        n = 3
    else:
        figsize = (12, 5)
        n = 2

    plt.figure(figsize=figsize)
    
    plt.subplot(1,n,1)
    plt.plot(xs, fs, 'b')
    plt.title('Original function')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    
    plt.subplot(1,n,2)
    plt.plot(xs, dfs, 'bo')
    plt.title('Graidient')
    plt.xlabel('x')
    plt.ylabel('df/dx')
    
    if d2f_dx2:
        plt.subplot(1,n,3)
        plt.plot(xs, d2fs, 'bo')
        plt.title('Hessian')
        plt.xlabel('x')
        plt.ylabel('d2f/dx2')

    plt.show()

def grad_trid_analytic(x):
    df1 = 2*(x - 1)
    x_pad1 = tf.concat([[0], x[:-1]], axis=0)
    x_pad2 = tf.concat([x[1:], [0]], axis=0)
    z = df1 - x_pad1 - x_pad2
    return z 
    
def check_same(a, b, eps=1e-6):
    """두 입력 텐서가 eps 값 이내로 같은지 판별"""
    a, b = tf.constant(a), tf.constant(b)
    diff = a - b
    test = tf.reduce_all(tf.abs(diff) < eps)
    if test:
        print(f"성공: 주어진 두 텐서가 eps={eps} 내에서 일치합니다.")
    else:
        print(f"실패: 주어진 두 텐서가 eps={eps} 이상의 차이를 가집니다.")

### 그래디언트 테이프

TensorFlow는 자동 미분을 위해 [`tf.GradientTape` API](https://www.tensorflow.org/api_docs/python/tf/GradientTape)를 제공합니다. `tf.GradientTape`는 일반적으로 `with ...` 구문의 형태로 (cf. [컨텍스트 매니저](https://docs.python.org/3/reference/compound_stmts.html#with)) 사용되며, 선언된 컨텍스트 안에서 watch하고 있는 텐서에 한하여 실행된 모든 연산을 테이프(tape)에 "기록"합니다. 그래디언트를 계산할 때는, 테이프에 기록된 연산을 역전파하는 `tape.gradient(output, input)` 함수를 사용합니다. 
* 학습가능한 변수 (trainable variable; `tf.Variable` 혹은 `tf.get_variable`을 통해 생성된 변수)는 자동으로 watch 합니다.
* 일반적인 `tf.Tensor` 는 `tape.watch()` 함수를 통해 수동으로 watch 할 수 있습니다.

아래 예제에서는 간단한 일변수 실수 함수인 $f(x)=x^2$의 도함수와 이계도함수를 TensorFlow 프레임워크를 통해 계산합니다. 출력된 plot을 통해서 실제로 미분이 잘 이루어졌음을 확인할 수 있습니다.

In [ ]:
# f(x)=x^2의 도함수와 이계도함수를 계산합니다.

def square(x):
    """입력의 제곱을 반환"""
    y = x * x
    return y

def grad_square(x):
    """square() 연산의 그래디언트 계산"""
    x = tf.constant(x, dtype=tf.float32)
    
    # tf.GradientTape을 통해 연산을 기록합니다.
    with tf.GradientTape() as t:
        # 미분 입력 연산을 watch 처리합니다.
        t.watch(x)
        # 대상 출력까지 연산을 수행합니다.
        y = square(x)
        
    return t.gradient(y, x)

def dgrad_square(x):
    """square() 연산의 이계도함수 계산"""
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        
        # GradientTape context를 중첩함으로써 
        # Higher-order differentiation을 수행할 수 있습니다.
        
        # x를 watch하는 상태에서 df_dx를 계산합니다.
        with tf.GradientTape() as t2:
            t2.watch(x)
            y = square(x)
        df_dx = t2.gradient(y, x)
    
    d2f_dx2 = t.gradient(df_dx, x)
    return d2f_dx2

# 정의한 세 함수를 (원함수, 도함수, 이계도함수) 시각화합니다.
plot_df_1d(square, grad_square, dgrad_square)

마찬가지 방식으로, 대부분의 TensorFlow 연산은 그 연산과정 내부에 그래디언트의 계산형식이 구현되어 있습니다. 역전파 알고리즘은 그래디언트 연산의 [chain rule](https://en.wikipedia.org/wiki/Chain_rule)을 통해 연쇄된 연산의 그래디언트를 취합하는 알고리즘입니다:
$$\frac{dz}{dx}=\frac{dz}{dy}\cdot \frac{dy}{dx}$$
아래 또 다른 예제에서는 Tensorflow 내장 연산인 `tf.cos()` 함수의 그래디언트 계산 과정을 보여줍니다.

In [ ]:
# f(x)=cos(x)의 도함수를 계산합니다.

def grad_cosine(x):
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        y = tf.cos(x)
    return t.gradient(y, x)

# 입력 텐서 x에 대한 f의 도함수
plot_df_1d(tf.cos, grad_cosine)

### Q1-1. Holder Table Function

이제 실제로 주어진 함수의 그래디언트를 직접 계산해봅시다. 아래 예시로 시각화된 Holder-table 함수는 최적화 알고리즘의 성능을 평가하는 용도로 사용되는 함수 중 하나입니다. 주어진 함수의 식을 참고하여, 아래 `grad_holder`를 구현해봅시다. 올바르게 구현되었다면, 아래 `check_same()` 테스트를 통과할 것입니다. 
* 이전 예시들과 달리, 이 함수의 입력은 __2차원__입니다. `x[0], x[1]` 와 같은 방식으로 각 인자에 접근할 수 있습니다.
* 그래디언트가 올바르게 계산되기 위해서는 TensorFlow에서 지원하는 연산만을 사용하여 함수를 구현해야 합니다. 
[본 링크](https://www.tensorflow.org/api_docs/python/tf/math)에서 TensorFlow가 지원하는 수학 연산자를 확인하고 사용해보시길 바랍니다.

<img src="resource/holder2.png">
<img src="resource/holder.png">

In [ ]:
def holder(x):
    y1 = tf.sin(x[0]) * tf.cos(x[1])
    y2 = tf.exp(tf.abs(1 - tf.norm(x) / np.pi))
    z = -tf.abs(y1 * y2)
    return z

def grad_holder(x):
    """Holder-table 함수의 입력 x = (x_1, x_2)에 대한 그래디언트를 계산합니다."""
    ##### 여기에 코드를 작성하세요 #####
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        y = holder(x)
    return t.gradient(y, x)

check_same(grad_holder([0.5, 0.5]), [-1.4660158, 0.70440483])
check_same(grad_holder([1.0, 0.5]), [-0.50257826, 0.96842515])
check_same(grad_holder([0.0, 0.0]), [0.0, 0.0])
check_same(grad_holder([-4.0, 0.0]), [1.1756195, 0.0])
check_same(grad_holder([0.0, -4.0]), [0.0, 0.0])

### Q1-2. Trid Function

이어서, 아래 식으로 주어진 Trid 함수에 대한 그래디언트를 구해봅시다. 이번에는, 함수의 입력으로 주어지는 `x`의 차원이 고정되어 있지 않을 것입니다. 따라서, 함수를 구현할 때 __`x`의 차원에 의존하지 않는 코드__를 작성하는 것이 중요합니다. TensorFlow에서 지원하는 다차원 연산을 활용하여 아래 `grad_trid` 함수를 구현해봅시다. 예를 들어, `tf.reduce_*(x)` 형태의 연산은 `x`의 차원에 관계없이 동작합니다.


<img src="resource/trid2.png">

In [ ]:
def trid(x):
    y1 = tf.reduce_sum((x - 1) ** 2)
    y2 = tf.tensordot(x[1:], x[:-1], 1)
    z = y1 - y2
    return z

def grad_trid(x):
    """Trid 함수의 입력 x = (x_1, ..., x_d)에 대한 그래디언트를 계산합니다."""
    ##### 여기에 코드를 작성하세요 #####
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        y = trid(x)
    return t.gradient(y, x)

for i in range(20):
    d = np.random.randint(5, 200)
    x = tf.random.normal([d])
    print(f"===== 시도 {i+1}: d={d} =====")
    computed = grad_trid(x)
    analytic = grad_trid_analytic(x)
    check_same(computed, analytic)

### Q1-3. Deep Neural Network

마지막으로, 일반적인 다층신경망 함수의 입력에 대한 그래디언트를 계산해봅시다. 아래 주어진 $f(x)$라는 3-layer 다층신경망을 사용해서 $\frac{df}{dx}$ 를 계산하는 함수를 구현해보세요. 출력되는 그래디언트의 형태가 입력과 같기 때문에 그래디언트 역시 이미지의 형태를 가집니다. 실제로 그래디언트가 어떤 형태로 나타나는지 시각화해서 확인해보세요. 실제 딥러닝 모델의 학습에서는 입력에 대한 그래디언트 대신 $f(x)$ 내부 연산의 가중치에 대한 그래디언트를 주로 계산하지만, 입력에 대한 그래디언트 또한 [적대적 입력 생성 (adversarial examples)](https://medium.com/@ml.at.berkeley/tricking-neural-networks-create-your-own-adversarial-examples-a61eb7620fd8) 등 딥러닝의 중요한 연구분야에서 널리 사용됩니다. 

In [ ]:
x = tf.random.normal([1, 28, 28])
f = keras.Sequential([
    L.Flatten(),
    L.Dense(256, activation=tf.nn.relu),
    L.Dense(256, activation=tf.nn.relu),
    L.Dense(256, activation=tf.nn.relu),
    L.Dense(1, activation=None)
])

def df_dx(x):
    ##### 여기에 코드를 작성하세요 #####
    with tf.GradientTape() as t:
        t.watch(x)
        y = f(x)
    return t.gradient(y, x)

# 출력된 그래디언트를 이미지 형태로 시각화합니다.
plt.imshow(df_dx(x).numpy().squeeze())

## Part 2: 다층 신경망을 통한 이미지 분류

본 실습의 Part 2에서는 강의에서 다룬 다층신경망을 활용하여 운동화나 셔츠 같은 옷 이미지를 분류하는 문제를 풀어볼 것입니다. 10개의 클래스와 70,000개의 흑백 이미지로 구성된 Fashion-MNIST 데이터셋을 사용하여 선형 로지스틱 모델, 신경망 모델, 그리고 컨볼루션 신경망 모델을 구성하고 학습한 뒤, 그들의 성능을 비교합니다. 아래 주어진 코드를 실행하면서 TensorFlow에서 신경망 모델이 어떤 방식으로 학습되는지 파악하시길 바랍니다.

In [ ]:
# 실습 과정에서 사용되는 도움함수를 정의합니다. 

def visualize_dataset(images, labels, class_names=None, cmap=plt.cm.binary):
    """데이터셋을 간략히 시각화함"""
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=cmap)
        if class_names:
            plt.xlabel(class_names[labels[i]])
        else:
            plt.xlabel(f"Label: {labels[i]}")
    plt.show()

In [ ]:
# Fashion-MNIST 데이터셋은 TensorFlow에서 바로 불러올 수 있습니다.
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# 각 데이터 샘플은 0~9 사이의 값을 라벨로 가지고, 각 라벨과 실제 대응되는 옷 종류는 아래와 같습니다.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# 불러온 데이터셋은 0~255 사이의 값을 가지는 흑백 이미지로 구성되어 있습니다.
# 네트워크의 안정적인 학습을 위해, 각 데이터가 0~1 사이의 값을 가지도록 전처리합니다.
# 이때, 훈련 데이터셋과 테스트 데이터셋을 같은 방식으로 전처리해주는 것이 중요합니다.
train_images = train_images / 255.0
test_images = test_images / 255.0

print(f'훈련 세트 수: {len(train_labels)}')
print(f'테스트 세트 수: {len(test_labels)}')
print('이미지 크기: %d x %d' % train_images.shape[1:])
print(f'레이블 형태: {train_labels}')

# 이미지, 라벨은 각각 numpy.array의 형태로 임포트됩니다.
# 효과적인 학습 구성을 위해, 1) 이미지, 라벨 쌍을 배치 형태로 묶는 것과 2) 각 배치를 tf.tensor로 변환하는 과정을 수행합니다.
# 이 과정은 tf.data.Dataset 높은 수준의 class를 통해서 쉽게 처리될 수 있습니다.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(64)

In [ ]:
# 불러온 Fashion-MNIST 데이터셋을 간략히 시각화합니다.
visualize_dataset(train_images, train_labels, class_names)

### 선형 로지스틱 모델 구성

우선, 가장 간단한 로지스틱 회귀 모델을 생성합니다. TensorFlow의 고수준 인터페이스 구현체인 `tf.keras`를 통해서 쉽게 선형 모델을 정의할 수 있습니다. 아래 정의된 `linear_model`은 `tf.keras`에서 제공되는 연산으로만 구성되었습니다:
* [`keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential): 연속된 레이어를 가진 네트워크를 표현합니다.
* [`keras.layers.Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten): Fashion-MNIST의 이미지는 28x28의 행렬 형태입니다. 선형 연산을 하기 위해, 해당 이미지를 1차원 텐서의 형태로 변환합니다. 
* [`keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): 입력에 `x` 대한 선형 변환을 수행합니다: `Wx+b`. (W, b)를 파라미터로 갖고, 이들 파라미터는 훈련 도중에 학습됩니다. Fully-connected Layer라고도 불립니다. 
* `activation=tf.nn.softmax`: 해당 `Dense` 레이어 연산 이후 softmax activation을 통과합니다.

`model.summary()`를 통해서 구성된 모델을 시각화하고, 총 몇 개의 파라미터가 사용되었는지 (`Total params`), 그 중 학습가능한 파라미터는 몇 개인지 (`Trainable params`) 등을 한 눈에 파악할 수 있습니다. `linear_model`의 경우는 총 7,850개의 파라미터를 사용하는 것을 확인하실 수 있습니다.

In [ ]:
linear_model = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(10, activation=tf.nn.softmax)
])
linear_model.summary()

### 선형 로지스틱 모델 학습

이제 구성된 `linear_model`을 훈련할 수 있습니다. `tf.keras`는 훈련 과정에 대해서도 매우 높은 수준의 API를 제공하지만, 여기서는 해당 API를 사용하는 대신 우선 학습의 세부적인 과정을 하나씩 살펴보도록 하겠습니다. 아래 코드를 대략적으로 요약하면 다음과 같습니다:
* `train_step()` 함수를 통해 모델의 파라미터를 업데이트 하고, `test_step()` 함수를 통해 모델의 성능을 평가합니다.
* `train_step()` 함수 내부에서는 `loss_object`를 Stochastic Gradient Descent (SGD)를 통해 최소화하고자 합니다.
* `optimizer`는 계산된 그래디언트를 업데이트에 적용하는 방식을 설정하고 수행합니다.
* `train_loss`, `train_accuracy`, `test_loss`, `test_accuracy`는 학습 도중 계산되는 성능 측정을 계산하기 위한 용도입니다.

학습 결과, 선형 로지스틱 모델을 통해 테스트 데이터셋에서 ~82% 정도의 정확성을 달성할 수 있다는 것을 확인할 수 있습니다.

In [ ]:
# 최적화하고자 하는 loss를 정의합니다.
loss_object = keras.losses.SparseCategoricalCrossentropy()

# SGD 알고리즘을 통해 최적화를 수행하고, learning rate를 0.1로 설정합니다.
optimizer = keras.optimizers.SGD(learning_rate=0.1)

# 학습 도중 계산되는 성능 측정값을 계산할 수 있는 연산자를 생성합니다.
train_loss = keras.metrics.Mean(name='train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = keras.metrics.Mean(name='test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
 
@tf.function
def train_step(model, images, labels):
    """한 번의 훈련 과정 업데이트를 구현"""
    
    # 파리미터에 대한 그래디언트를 구하기 위해 tf.GradientTape을 기록합니다.
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    # GradientTape에서 기록한 연산에 대한 그래디언트를 계산합니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 계산된 그래디언트를 통해 모델을 업데이트 합니다.
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, images, labels):
    """한 번의 테스트 과정을 구현"""
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
# 전체 데이터셋을 5번 재방문 할 때까지 훈련을 반복합니다.
EPOCHS = 5

for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_ds):
        train_step(linear_model, images, labels)
        
        # 매 100 step마다 훈련 데이터셋에 대한 성능을 측정합니다.
        if i % 100 == 99:
            print(f"TRAIN: Epoch {epoch+1}, Step: {i+1}, "
                  f"Loss: {train_loss.result():.4f}, Accuracy: {train_accuracy.result():.4f}")

    for test_images, test_labels in test_ds:
        test_step(linear_model, test_images, test_labels)
    print ("TEST: Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}".format(epoch+1, test_loss.result(), 
                                                                   test_accuracy.result()))

### Keras API: `model.compile()`과 `model.fit()`

위 훈련 과정은 일반적인 신경망 학습 과정에서 비슷하게 사용되기 때문에, `tf.keras`에서는 해당 과정에 대한 고수준 API를 제공합니다: 
* `model.compile()`: 해당 모델을 어떻게 훈련할지 설정합니다.
* `model.fit()`: `compile`한 설정을 바탕으로 주어진 데이터셋에 대한 학습을 수행합니다.

아래 예시 코드처럼, 두 API를 사용하면 위에서 진행한 훈련 과정을 훨씬 간단하게 시작할 수 있습니다. 이후 코드에서는 편의 상 아래 방식으로 훈련을 수행할 것입니다.

In [ ]:
linear_model = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(10, activation=tf.nn.softmax)
])
linear_model.summary()

linear_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1),
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

print()
print("===== TRAIN =====")
history = linear_model.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = linear_model.evaluate(test_ds)

### 2-레이어 신경망 모델

선형 로지스틱 모델을 학습하여 ~82% 정도의 정확성을 가진 모델을 얻을 수 있었습니다. 이제, 선형 모델에서 레이어를 더하여 비-선형 신경망 모델을 만들고 학습시켜봅시다. 아래 `nn_model`은 위 `linear_model`과 유사하게 구성되었지만, ReLU activation을 가지고 128개의 뉴런을 가지는  `Dense` 레이어가 추가되었고 총 101,770개의 파라미터를 가집니다. 선형 모델을 학습할 때와 같은 방법으로 `nn_model` 또한 학습할 수 있습니다. 실제로 학습을 수행하여, 선형 로지스틱 모델에 비해 정확성이 향상되었는지 확인해 보십시오.

In [ ]:
nn_model = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(128, activation=tf.nn.relu),
    L.Dense(10, activation=tf.nn.softmax)
])
nn_model.summary()

nn_model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy', 
                 metrics=['accuracy'])

print()
print("===== TRAIN =====")
history = nn_model.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = nn_model.evaluate(test_ds)

### 컨볼루션 신경망 모델

컨볼루션 신경망 모델 또한 `tf.keras`를 통해 쉽게 구성될 수 있습니다. 아래 예시로 주어진 `cnn_model`은 2개의 컨볼루션 레이어를 가지고 있습니다. 
`nn_model`과 비교했을 때, `Conv2D` 레이어는 3차원 형태로 연산을 수행합니다: `(height, width, channel)`. 단, 마지막 `Dense` 레이어 직전에는 `Flatten()`을 통해 1차원 형태로 변환해줍니다.
마찬가지로 학습을 수행하여 기존 모델 대비 정확성이 향상되는지 확인해 보시길 바랍니다.

In [ ]:
cnn_model = keras.Sequential([
    L.Reshape((28, 28, 1), input_shape=(28, 28)),
    L.Conv2D(64, 3, activation=tf.nn.relu),
    L.Conv2D(32, 3, activation=tf.nn.relu),
    L.Flatten(),
    L.Dense(10, activation=tf.nn.softmax)
])
cnn_model.summary()

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

print()
print("===== TRAIN =====")
history = cnn_model.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = cnn_model.evaluate(test_ds)

## Q2. 파라미터 효율적인 신경망

위 예시에서는 선형 로지스틱 모델, 신경망 모델, 그리고 컨볼루션 신경망 모델로 구성을 변화시킴으로써 주어진 문제에 대한 정확성을 더 높일 수 있다는 것을 확인했습니다. 신경망 (및 컨볼루션 신경망) 모델은 선형 로지스틱 모델에 비해 높은 정확성을 도달할 수 있지만, 대신 훨씬 많은 수의 파라미터를 사용해야 한다는 것도 발견할 수 있었습니다. 

|             | `linear_model` | `nn_model` | `cnn_model` |
|:-----------:|:--------------:|:----------:|:-----------:|
| 파라미터 수 |      7,850     |   101,770  |   203,434   |

모델 파라미터의 수는 모델의 연산 효율성과 연결됩니다: 적은 파라미터를 가질수록, 실제 사용에서 더욱 빠르게 연산을 처리할 수 있는 경향이 있습니다. 따라서, 위에서 구성한 신경망 모델은 "파라미터 수 대비 정확도"의 관점에서 누군가에게는 큰 효용성이 없는 모델이 될 수도 있습니다.

### Q2-1. `nn_model` vs `cnn_model`

그렇다면, "파라미터 수 대비 정확도"의 관점에서 보았을 때 `nn_model`과 `cnn_model` 중 더 효율적인 아키텍쳐는 무엇일까요? 아래 주어진 코드에서, `nn_model_large`의 hidden neuron의 갯수를 조절해서 2-레이어 신경망 모델이지만 `cnn_model`과 비슷한 파라미터 수를 가지게끔 구성한 뒤 학습하고 그 성능을 `cnn_model`과 비교해보세요. 주어진 2-레이어 신경망에서, hidden neuron의 갯수를 늘리는 것이 컨볼루션 신경망의 장점을 뛰어넘을 수 있나요? 그렇지 않다면, 다른 방법을 사용하거나 파라미터를 더 사용하더라도 2-레이어 신경망이 `cnn_model`의 성능을 뛰어넘을 수 있는 방법이 있을까요?

In [ ]:
nn_model_large = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(256, activation=tf.nn.relu),
    L.Dense(10, activation=tf.nn.softmax)
])
nn_model_large.summary()

nn_model_large.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])

print()
print("===== TRAIN =====")
history = nn_model_large.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = nn_model_large.evaluate(test_ds)

### Q2-2. 효율적인 신경망 설계하기 

이제 직접 신경망과 학습 과정을 설계하여 최대한 파라미터-효율성이 높은 신경망을 찾아봅시다. 새로운 신경망 모델 `my_model`을 설계하되, 다음과 같은 제약 사항이 있습니다:
1. __`my_model`의 테스트 성능은 `cnn_model`의 성능과 비슷하거나 (~91%) 좋아야합니다.__
2. __`my_model`의 총 파라미터 갯수는 적으면 적을수록 좋습니다.__

하나의 예시로서, 아래 주어진 `cnn_model_v2`를 학습해보세요: 해당 모델은 `cnn_model`에 `MaxPool2D` 레이어를 추가함으로써, `nn_model` 보다 적은 파라미터를 사용하면서 성능은 유지할 수 있도록 구성되었습니다. 

강의에서 다룬 내용을 최대한 활용하여, 모델의 파라미터 효율성을 개선해보시길 바랍니다. 다음과 같은 옵션이 존재할 수 있고, 다른 방법도 가능합니다.
* __모델 아키텍쳐 ([`tf.keras.layers`](https://www.tensorflow.org/api_docs/python/tf/keras/layers))__: 풀링 레이어를 활용하거나 레이어의 뉴런 수를 조절하는 등으로 불필요한 파라미터를 절약할 수 있습니다. 레이어를 늘리면 파라미터 수도 늘어나지만, 뉴런 수를 조절하면 성능 향상의 효과가 더 클수도 있습니다. `my_model.summary()`를 통해 레이어의 입력, 출력 형태를 잘 파악하고 적절한 레이어를 배치하는 것이 중요합니다.
* __활성화 함수 ([`tf.keras.activations`](https://www.tensorflow.org/api_docs/python/tf/keras/activations)) 변경__: 적절한 활성화 함수는 학습을 더 안정적으로 만들어 주고 모델 성능에도 영향을 줄 수 있습니다.
* __최적화 방식 조정 ([`tf.keras.optimizers`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))__: 최적화 알고리즘이나 learning rate 스케쥴링에 따라서 학습이 더욱 개선될 수 있습니다.
* __학습을 더 오래 하기, 중간에 학습 멈추기__: 어떤 경우는 학습이 충분히 되지 않아서 해당 모델의 성능을 충분히 활용하지 못하기도 합니다. 한편, 너무 오래 학습하는 경우 과적합이 (over-fitting) 심해지는 경우도 있습니다.

In [ ]:
cnn_model_v2 = keras.Sequential([
    L.Reshape((28, 28, 1), input_shape=(28, 28)),
    L.Conv2D(64, 3, activation=tf.nn.relu),
    L.Conv2D(32, 3, activation=tf.nn.relu),
    L.MaxPool2D(),
    L.Flatten(),
    L.Dense(10, activation=tf.nn.softmax)
])
cnn_model_v2.summary()

cnn_model_v2.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

print()
print("===== TRAIN =====")
history = cnn_model_v2.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = cnn_model_v2.evaluate(test_ds)

In [ ]:
##### 여기에 코드를 작성하세요 #####

my_model = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(10, activation=tf.nn.softmax)
])
my_model.summary()

my_model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy', 
                 metrics=['accuracy'])

##### 여기에 코드를 작성하세요 #####

print()
print("===== TRAIN =====")
history = my_model.fit(train_ds, epochs=5)
print("===== TEST =====")
test_loss, test_acc = my_model.evaluate(test_ds)